In [ ]:
import os 
from time import sleep

from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import datetime
import glob
from random import random

# У меня были размечены фотографии под каждую квартиру
# Все фото лежали в одной диреткории
# За скриптом для разметки обращайтесь в TG @nikmedoed
PHOT_PATH = r"E:/Решения/"

# P.S. есть ещё скрипт для рассылки по телеграм, 
# а также сбору информации из групповых чатов 
# и другие полезные для проведения групповой работы скрипты, пишите

In [ ]:
# Назначение опций:
# создание профиля пользователя для последующего входа в мессенджер без авторизации

options = webdriver.ChromeOptions()
options.add_argument('--allow-profiles-outside-user-dir')
# options.add_argument('--enable-profile-shortcut-manager')
options.add_argument(f'user-data-dir={os.getcwd()}\\User')
options.add_argument("--start-maximized");
# options.add_argument(f'--profile-directory="Default2"')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=options) 
browser.get("https://web.whatsapp.com/")

In [ ]:
# Это Xpaths до элементов, они могут меняться

PATHS = {
    "users": '//*[@id="pane-side"]/div[1]/div/div/div',
    # карточки пользователей идут в случайном порядке, надо проверять имя
    
    'username':".//div/div/div[2]/div[1]/div[1]/span",
    # имя относительно элемента "карточка (бейдж) пользователя"
    
    'search':'//*[@id="side"]/div[1]/div/label/div/div[2]',
    # путь до поля "поиск"
    
    'message':'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[2]',
    # путь до текстового поля, в которое можно отправлять символы
    
    'file': '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div'
    # путь до исконки файла, по ней нужно кликнуть, чтобы в итоге появился целевой input
}

In [ ]:
def log_error(text):
    print(text)
    with open("log_err", "a+") as f:
        f.write(f"{text}\n")        
        
def get_card_name(user):
    name = ""
    try:
        name = user.find_element_by_xpath(PATHS['username']).text
    except:
        pass
    return name

# очищается текстовое поле, как поиск, так и любое другое, с учётом лага на анимации
def fill_text(element):
    element.click()
    sleep(0.5)
    element.clear()
    return element

def search_user(browser, name):
    search =  browser.find_element_by_xpath(PATHS['search'])
    fill_text(search).send_keys(name)
    sleep(1)
    users = browser.find_elements_by_xpath(PATHS['users'])
    return [user for user in users if get_card_name(user).startswith(name)]

#### Внимание!!!
Далее начинается рассылка сообщений. Сначала стоит проверить, всё ли впорядке. Код для проверки ниже

In [ ]:
notificate = open("kv_maybe.txt","r", encoding="utf-8").read().split("\n")

for n, kv in enumerate(notificate):

    # сообщение лучше перечитывать каждый раз, чтобы можно было находу править ошибки
    # я использовал разделитель, чтобы можно было слать неодним сообщением
    # после определённого числа символов часть сообщения скрывается под "далее"
    message = open("messwa.txt","r", encoding="utf-8").read().split("=====")

    # Я проверял, есть ли у меня вообще фото для отправки, иначе не надо было писать
    photos = glob.glob(f"{PHOT_PATH}{kv}-*")
    if photos:
        
        # у меня произошёл некоторый косяк с базой контактов 
        # и некоторые запоминились в мессенджере под старыми именами
        target_users = search_user(browser, f"ГРД - {kv} - ")
        if not target_users:
            target_users = search_user(browser, f"ГРД1 - {kv} - ")            
        if target_users:
            user = target_users[0]
            user.click()

            print(f"{kv} :: start {get_card_name(user)}")

            sleep(3+random()*5)
            try:
                # если убрать два блока  отправки, то можно собрать список тех, у кого есть аккаунт

                # блок отправки файлов
                file_icon = browser.find_element_by_xpath(PATHS['file'])
                file_icon.click()
                for photo in photos:    
                    file_input = [ i for i in browser.find_elements_by_xpath('//input')][0]
                    file_input.send_keys(photo)
                    sleep(3)                  
                ActionChains(browser).send_keys(Keys.ENTER).perform() 
                sleep(1)

                # блок отправки сообщений
                message_box = browser.find_element_by_xpath(PATHS['message'])
                fill_text(message_box)
                for mes in message:
                    for part in mes.strip().split('\n'):
                        message_box.send_keys(part)
                        ActionChains(browser).key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(
                            Keys.ENTER).perform()
                    message_box.send_keys('\n')

                print(f"ok {n}")
                with open("log_notificated.txt", "a+") as log_ok:
                    log_ok.write(f"{kv}\n")
            except Exception as e:
                log_error(f"{kv} - error, err: {e}")
        else:
            log_error(f"{kv} - not found")
    else:
        log_error(f"{kv} no photo")

### Это код для проверки корректности работы скрипта
А вернее корректности путей до элементов и работоспособности

In [ ]:
# это проверит поиск пользователя

name = f"Тома"

target_users = search_user(browser, name)
if target_users:
    user = target_users[0]
    user.click()
    sleep(1)

print(user.text)

In [ ]:
# Это сделает рассылку по открытому диалогу
# я открывал диалог с собой и проверял работу отправки перед рассылкой нового сообщения

kv = "5"

message = open("messwa.txt","r", encoding="utf-8").read().split("=====")
photos = glob.glob(f"{PHOT_PATH}{kv}-*")

file_icon = browser.find_element_by_xpath(PATHS['file'])
file_icon.click()
for photo in photos:    
    file_input = [ i for i in browser.find_elements_by_xpath('//input')][0]
    file_input.send_keys(photo)
    sleep(3)                  
ActionChains(browser).send_keys(Keys.ENTER).perform() 
sleep(1)


message_box = browser.find_element_by_xpath(PATHS['message'])
fill_text(message_box)
for mes in message:
    for part in mes.strip().split('\n'):
        message_box.send_keys(part)
        ActionChains(browser).key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(
            Keys.ENTER).perform()
    message_box.send_keys('\n')